__Machine Learning in Business__

You work for the OilyGiant mining company. Your task is to find the best place for a new well.

You have data on oil samples from three regions. Parameters of each oil well in the region are already known.

Build a model that will help to pick the region with the highest profit margin.

Analyze potential profit and risks using the Bootstrapping technique.


importing the libraries i'll need:

In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics
#from sklearn.preprocessing import LabelEncoder
#from sklearn import svm
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
#from sklearn.metrics import recall_score
#from sklearn.metrics import precision_score
#from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils import shuffle
#from sklearn.metrics import confusion_matrix
#import seaborn as sns
#import sidetable as stb
#import matplotlib.pyplot as plt
#from sklearn.metrics import f1_score
#from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression

# Download and prepare the data. Explain the procedure

In [3]:
try:
    
    path='C:\\Users\\aviv\\Downloads\\geo_data_0.csv'
    data_0 = pd.read_csv(path)
    print(data_0.head())
    
    path='C:\\Users\\aviv\\Downloads\\geo_data_1.csv'
    data_1 = pd.read_csv(path)
    print(data_1.head())
    
    path='C:\\Users\\aviv\\Downloads\\geo_data_2.csv'
    data_2 = pd.read_csv(path)
    print(data_2.head())

    
except:
    path = '/datasets/geo_data_0.csv'
    data_0 = pd.read_csv(path)
    print(data_0.head())
    
    path = '/datasets/geo_data_1.csv'
    data_1 = pd.read_csv(path)
    print(data_1.head())
    
    path = '/datasets/geo_data_2.csv'
    data_2 = pd.read_csv(path)
    print(data_2.head())
    

    

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [4]:
data_list=[data_0, data_1, data_2]
for data in data_list:
    print(data.info())
    print(data.describe(include='all'))

    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
            id             f0             f1             f2        product
count   100000  100000.000000  100000.000000  100000.000000  100000.000000
unique   99990            NaN            NaN            NaN            NaN
top      fiKDv            NaN            NaN            NaN            NaN
freq         2            NaN            NaN            NaN            NaN
mean       NaN       0.500419       0.250143       2.502647      92.500000
std        NaN       0.871832       0.504433       3.248248      44.288691
min        NaN      -1.408605      -

In [5]:
for data in data_list:
    data.drop("id", axis=1, inplace = True)
    print(data.info())
    print(data.describe(include='all'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
<class 'pandas.cor

# Train and test the model for each region:

## Split the data into a training set and validation set at a ratio of 75:25.

In [6]:
#geo0
features0 = data_0.drop(['product'], axis=1)
target0 = data_0['product']
features_train0, features_valid0, target_train0, target_valid0 = train_test_split(
    features0, target0, test_size=0.25, random_state=12345)

In [7]:
#geo1
features1 = data_1.drop(['product'], axis=1)
target1 = data_1['product']
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    features1, target1, test_size=0.25, random_state=12345)

In [8]:
#geo2
features2 = data_2.drop(['product'], axis=1)
target2 = data_2['product']
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    features2, target2, test_size=0.25, random_state=12345)

## Train the model and make predictions for the validation set.

In [9]:
#geo0
model0 = LinearRegression()# initialize model constructor
model0.fit(features_train0, target_train0) # train model on training set



LinearRegression()

In [10]:
#geo1
model1 = LinearRegression()# initialize model constructor
model1.fit(features_train1, target_train1) # train model on training set

LinearRegression()

In [11]:
#geo2
model2 = LinearRegression()# initialize model constructor
model2.fit(features_train2, target_train2) # train model on training set

LinearRegression()

## Save the predictions and correct answers for the validation set.

In [12]:
predictions_valid0 = model0.predict(features_valid0)# get model predictions on validation set

In [13]:
predictions_valid1 = model1.predict(features_valid1) # get model predictions on validation set

In [14]:
predictions_valid2 = model2.predict(features_valid2) # get model predictions on validation set

## Print the average volume of predicted reserves and model RMSE.

In [15]:
result0 = mean_squared_error(target_valid0,predictions_valid0)**0.5 # calculate RMSE on validation set
print("RMSE of the linear regression model on the validation set:", result0)
print("avg volume of predicted reserves:", predictions_valid0.mean())

predicted_valid0 = pd.Series(target_train0.mean(), index=target_valid0.index)
print("RMSE for constant model:",mean_squared_error(predicted_valid0, target_valid0)**0.5)
print("avg volume predicted by constant model:", predicted_valid0.mean())

RMSE of the linear regression model on the validation set: 37.5794217150813
avg volume of predicted reserves: 92.59256778438035
RMSE for constant model: 44.289591053907365
avg volume predicted by constant model: 92.64046775311564


In [16]:
result1 = mean_squared_error(target_valid1,predictions_valid1)**0.5 # calculate RMSE on validation set
print("RMSE of the linear regression model on the validation set:", result1)
print("avg volume of predicted reserves:", predictions_valid1.mean())
predicted_valid1 = pd.Series(target_train1.mean(), index=target_valid1.index)
print("RMSE for constant model:",mean_squared_error(predicted_valid1, target_valid1)**0.5)
print("avg volume predicted by constant model:", predicted_valid1.mean())

RMSE of the linear regression model on the validation set: 0.893099286775617
avg volume of predicted reserves: 68.72854689544602
RMSE for constant model: 46.02144533725471
avg volume predicted by constant model: 68.85895465855572


In [17]:
result2 = mean_squared_error(target_valid2,predictions_valid2)**0.5 # calculate RMSE on validation set
print("RMSE of the linear regression model on the validation set:", result2)
print("avg volume of predicted reserves:", predictions_valid2.mean())
predicted_valid2 = pd.Series(target_train2.mean(), index=target_valid2.index)
print("RMSE for constant model:",mean_squared_error(predicted_valid2, target_valid2)**0.5)
print("avg volume predicted by constant model:", predicted_valid2.mean())

RMSE of the linear regression model on the validation set: 40.02970873393434
avg volume of predicted reserves: 94.96504596800489
RMSE for constant model: 44.90234968510567
avg volume predicted by constant model: 95.03858906373009


## Analyze the results.
second region has smallest RMSE, which means we can predict most accurately the volumes. so even though the volumes are smallest in 2nd region  we can be more sure of their accuracy

# Prepare for profit calculation:

## Store all key values for calculations in separate variables.

In [18]:
UNIT_PRICE = 4500 # a unit is 1000 barrels
BUDGET_200 = 10**8 # budget is 100 million $
N_WELLS = 200

## Calculate the volume of reserves sufficient for developing a new well without losses. Compare the obtained value with the average volume of reserves in each region.

In [21]:
BUDGET_PER_WELL = BUDGET_200 / N_WELLS
BUDGET_PER_WELL

500000.0

now we have to figure out how many units a well has to have to develop without losses:

In [22]:
MIN_VOLUME = BUDGET_PER_WELL / UNIT_PRICE
MIN_VOLUME

111.11111111111111

## Provide the findings about the preparation for profit calculation step.

so a well has to have at least 111 units (1000 barrels) to not lose.
the avg volume predicted by the model for each of the regions was 
- geo0 - 92
- geo1 - 69
- geo2 - 94

so taking only these numbers into account, i wouldn't pick geo1 to develop wells in, as the chances of being 200 wells with 111 units is slimmer because the avg amount is lower.

# Write a function to calculate profit from a set of selected oil wells and model predictions:

##  Pick the wells with the highest values of predictions. 

sorting the predictions by volume, than taking the 200 most voluminous (predicted )wells

In [23]:
top_200_a = pd.Series(predictions_valid0).sort_values(ascending=False).head(200)
top_200_a



9317     180.180713
219      176.252213
10015    175.850623
11584    175.658429
23388    173.299686
            ...    
7888     148.507064
7890     148.481767
24051    148.476498
24160    148.436761
20340    148.365941
Length: 200, dtype: float64

In [24]:
top_200_b = pd.Series(predictions_valid1).sort_values(ascending=False).head(200)
top_200_b


20430    139.818970
7777     139.773423
8755     139.703330
1178     139.560938
4285     139.516754
            ...    
12774    138.421423
8040     138.416960
2257     138.413881
17108    138.412834
9693     138.410697
Length: 200, dtype: float64

In [25]:
top_200_c = pd.Series(predictions_valid2).sort_values(ascending=False).head(200)
top_200_c

22636    165.856833
24690    165.679685
7811     163.439962
1581     162.062589
6751     161.797476
            ...    
5541     142.490763
18126    142.485922
11122    142.465777
22669    142.454763
23447    142.408851
Length: 200, dtype: float64

##  Summarize the target volume of reserves in accordance with these predictions

Here you need to calculate minimum volume of reserves sufficient for developing a new well without losses.

so as we calculated in chapter 3, the minimum volume of reserves to develop a well without losses is 111 units.

## Provide findings: suggest a region for oil wells' development and justify the choice. Calculate the profit for the obtained volume of reserves.

Here you need to create a function to calculate the potential profit from the top 200 wells by region.

In [27]:
def profit(well_list):
    print("profit of region: ", (sum(well_list)*UNIT_PRICE - BUDGET_200))
    return
profit(top_200_a)
profit(top_200_b)
profit(top_200_c)

    

profit of region:  39960488.77465123
profit of region:  24857120.519735396
profit of region:  33217543.962432653


although geo1 is predicted to be the least voluminous and profitable, i'd take a chance with that region as the model has the smallest RMSE, so we can trust that geo1 actually delivers the goods even though it may seem the least profitable region

# Calculate risks and profit for each region:

## Use the bootstrapping technique with 1000 samples to find the distribution of profit.
## Find average profit, 95% confidence interval and risk of losses. Loss is negative profit, calculate it as a probability and then express as a percentage.

In [37]:
def boot_strap(target, predictions):
    def profit(target, probabilities, count):
        probs_sorted = pd.Series(probabilities).sort_values(ascending=False)
        selected = target[probs_sorted.index][:count]
        
        well_boot = (selected*UNIT_PRICE) - BUDGET_PER_WELL
        #print(well_boot)
        return well_boot.sum()

    state = np.random.RandomState(12345)
    n_tests = 1000
    ##experiment with indices
    predictions = pd.Series(predictions, index=target.index)
    
    ##
    values = []
    for i in range(n_tests):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        sample = profit(target_subsample, probs_subsample, 200)
        values.append(sample)
    
    loss = 0
    for val in values:
        if val < 0:
            loss += 1
    values = pd.Series(values)
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    mean = values.mean()
    print("Average profit:", mean/10**6,"million dollars")
    print("2.5% quantile:", lower/10**6, "million dollars")
    print("97.5% quantile:", higher/10**6, "million dollars")
    print('Number of losses:', loss)
    print("percent losses:", loss / n_tests * 100, "%")

In [38]:
print("region 1:")
boot_strap(target_valid0, predictions_valid0)
print('*'*50)
print("region 2:")
boot_strap(target_valid1, predictions_valid1)
print('*'*50)
print("region 3:")
boot_strap(target_valid2, predictions_valid2)

region 1:
Average profit: 4.259385269105927 million dollars
2.5% quantile: -1.0209009483793727 million dollars
97.5% quantile: 9.479763533583691 million dollars
Number of losses: 60
percent losses: 6.0 %
**************************************************
region 2:
Average profit: 5.1522277344328975 million dollars
2.5% quantile: 0.6887322537050338 million dollars
97.5% quantile: 9.315475912570488 million dollars
Number of losses: 10
percent losses: 1.0 %
**************************************************
region 3:
Average profit: 4.3500836278275585 million dollars
2.5% quantile: -1.2888054732978877 million dollars
97.5% quantile: 9.69706954180266 million dollars
Number of losses: 64
percent losses: 6.4 %


## Provide findings: suggest a region for development of oil wells and justify the choice.

for region 2:
- the average profit is higher than the other 2 regions
- the accuracy for predictions is high (by RMSE)
- the risk is low
- we can be 95% it'll be at least 0.6 million dollars profitable
for the reasons above, i suggest choosing region 2 (geo 1)